## Code infrastructure for importing CSV data and initializing imported data types
JDL/Data Delve Engineer LLC 12/22/20

This notebook and its imported libraries give a code framework for dealing well with imported CSV files in a project.  The code demos use of a Column Info Class Object that contains the metadata about project variables.  In this case, the key metadata is each variable's data type.<br />

Background links
* https://stackoverflow.com/questions/47988770/why-applytype-get-inconsistent-results-in-pandas
* https://stackoverflow.com/questions/52622708/why-do-i-get-different-results-for-pandas-series-apply-and-dataframe-applymap
* https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html
* https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html
* https://stackoverflow.com/questions/54906818/pandas-read-csv-with-dtypes-but-mixed-type-columnsna-values

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

#Import utility modules
import colinfo
import pdutil

#### Create sample data with various data types and write to CSV
* `np.nan` creates blanks in the CSV file
* For Flag columns (e.g. flagging that an event happened in time-series data), a 1/blank format is most efficient for CSV filesize, but Boolean is better for RAM and Pyarrow Feather file size efficiency
* Feather is a preferred internal file format because of its file size efficiency versus CSV and also because feather stores data type for columns/variables
* Feather insists on consistent data type within a variable (mixed types in a column 

In [2]:
dftemp = pd.DataFrame({'dt_col': ['1/1/2020',np.nan,'1/3/2020','1/4/2020'], 'int_col':[4,np.nan,3,1],
                      'float_col':[0.0,1.0,np.nan,4.5],'flag_col':[1,np.nan,np.nan,1],'text_col':['a','b',None,'d']})

#Write to CSV (to be read back in to fully simulate CSV behavior with missing values etc.)
dftemp.to_csv('MixedTypes.csv', index=False)

#### Read column info CSV file containing metadata about the data columns for the project

In [3]:
tblCI = colinfo.tblcolinfo()
tblCI.dfcolinfo = pd.read_csv(tblCI.file, index_col=tblCI.name)
tblCI = colinfo.BuildTypeAndFlagDicts(tblCI)

print('\ncolinfo data type and isflagcol dictionaries\n\n', tblCI.dict_types)
print(tblCI.dict_isflagcol, '\n\ncolinfo DataFrame\n')
tblCI.dfcolinfo


colinfo data type and isflagcol dictionaries

 {'dt_col': 'dt', 'int_col': 'Int16', 'float_col': 'float', 'flag_col': 'bool', 'text_col': 'object'}
{'dt_col': False, 'int_col': False, 'float_col': False, 'flag_col': True, 'text_col': False} 

colinfo DataFrame



,description,units,type
name,,,
dt_col,A Datetime column,NaN,dt
int_col,An integer column,NaN,Int16
float_col,A float column,NaN,float
flag_col,A flag column - stored as 1/blank format in CSV,NaN,bool_flag
text_col,A text column,NaN,object


### To Open an Individual CSV data File:
* call pdutil.ImportCSV()
  * opens the file and sets data types per colinfo dtype
  * ImportCSV also converts bool_flag columns from 1/blank to Boolean

In [4]:
df = pdutil.ImportCSV('MixedTypes.csv', tblCI)
df

,dt_col,int_col,float_col,flag_col,text_col
0,2020-01-01,4,0.0,True,a
1,NaT,<NA>,1.0,False,b
2,2020-01-03,3,NaN,False,NaN
3,2020-01-04,1,4.5,True,d


In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   dt_col     3 non-null      datetime64[ns]
 1   int_col    3 non-null      Int16         
 2   float_col  3 non-null      float64       
 3   flag_col   4 non-null      bool          
 4   text_col   3 non-null      object        
dtypes: Int16(1), bool(1), datetime64[ns](1), float64(1), object(1)
memory usage: 438.0 bytes


#### Feather write/read test

In [6]:
df.to_feather('ftest.feather')
df2 = pd.read_feather('ftest.feather')
df2

,dt_col,int_col,float_col,flag_col,text_col
0,2020-01-01,4,0.0,True,a
1,NaT,<NA>,1.0,False,b
2,2020-01-03,3,NaN,False,None
3,2020-01-04,1,4.5,True,d


## A couple of insights for diagnostics on data types
* `pd.isnull()` can be used to isolate missing values for numeric columns
* `applymap(type)` shows element-wise data types
* data type comparisons are facilitated by using `.applymap.iloc[x]` instead of `.iloc[x].apply(type)` that gives Numpy data types

In [7]:
dftemp.isnull()

,dt_col,int_col,float_col,flag_col,text_col
0,False,False,False,False,False
1,True,True,False,True,False
2,False,False,True,True,True
3,False,False,False,False,False


In [8]:
df.applymap(type)

,dt_col,int_col,float_col,flag_col,text_col
0,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'int'>,<class 'float'>,<class 'bool'>,<class 'str'>
1,<class 'pandas._libs.tslibs.nattype.NaTType'>,<class 'pandas._libs.missing.NAType'>,<class 'float'>,<class 'bool'>,<class 'str'>
2,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'int'>,<class 'float'>,<class 'bool'>,<class 'float'>
3,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'int'>,<class 'float'>,<class 'bool'>,<class 'str'>


In [9]:
print(df.applymap(type).iloc[0], '\n\n')
print(df.iloc[0].apply(type))

dt_col       <class 'pandas._libs.tslibs.timestamps.Timesta...
int_col                                          <class 'int'>
float_col                                      <class 'float'>
flag_col                                        <class 'bool'>
text_col                                         <class 'str'>
Name: 0, dtype: object 


dt_col       <class 'pandas._libs.tslibs.timestamps.Timesta...
int_col                                  <class 'numpy.int16'>
float_col                              <class 'numpy.float64'>
flag_col                                 <class 'numpy.bool_'>
text_col                                         <class 'str'>
Name: 0, dtype: object


#### Mixed types within a column don't work with feather
* Feather sets type by column --not element-wise
* Create mixed type within int_col by reading the CSV with dtype='object' and then inserting an integer series for some values

In [10]:
#Read in a fresh DataFrame from the CSV
df_in = pd.read_csv('MixedTypes.csv', dtype='object') 

#Insert a mis-typed snippet into df['int_col']
df_in.loc[range(2,4),'int_col'] = pd.Series([1,5], index=range(2,4))

#Drop unrelated columns
df3 = df_in.drop(['dt_col', 'float_col', 'flag_col', 'text_col'], axis=1)


df3.applymap(type)

,int_col
0,<class 'str'>
1,<class 'float'>
2,<class 'int'>
3,<class 'int'>


In [11]:
#Raises: "ArrowTypeError: ("Expected a bytes object, got a 'int' object", 'Conversion failed for column int_col with type object')"
#df3.to_feather('ftest2.feather')